<h1>心理状态分析V1</h1>
<ul>
<li><h3>基于LSTM</h3></li>
<li><h3>微博评论数据集</h3></li>
<li><h3>标记negative(0)与positive(1)</h3></li>
</ul>

<h2>1.读取数据</h2>
<h3>1.1导入工具包</h3>

In [ ]:
import re
from bs4 import BeautifulSoup
import pandas as pd
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import jieba

<h3>1.2读取数据</h3>
<h4>读取训练数据</h4>

In [ ]:
mydata = pd.read_csv(r'.\data\weibo_senti_100k.csv') #导入微博评论数据
train_csv = mydata[12000:-12000]
train_csv = train_csv.reset_index()
train_csv

<h4>读取测试数据</h4>

In [ ]:
test_csv = mydata[:12000].append(mydata[-12000:]) #划分测试集
test_csv = test_csv.reset_index()
test_csv

<h2>2.数据预处理</h2>
<h3>2.1评论文字预处理(规范化)</h3>
<h4>定义评论文字预处理函数</h4>

In [ ]:
def review_to_wordlist(review):
    '''对每条review中的不合法字符进行替换，并进行分词转化成词表'''
    review_text = BeautifulSoup(review, "html.parser").get_text()
    review_text = re.sub('@[\u4e00-\u9fa5]*|#(.)*#'," ", review_text) #先去除用户名和话题标签
    review_text = re.sub('[^\u4e00-\u9fa5]','',review_text) #再去除标点和英文
    review_text = jieba.lcut(review_text) #分词
    return review_text

In [ ]:
#对训练集进行处理
train_label = train_csv['label'] #读取训练集中的'label'列
y_train=train_label.values #将'label'列由series结构转换为ndarray结构
train_texts = [] #二维列表，每个元素都是一条文本分词后的词语列表
for i in range(len(train_csv['review'])):
    train_texts.append(' '.join(review_to_wordlist(train_csv['review'][i]))) #逐个添加每条文本分词后的词语列表
#显示示例
print('评论内容：')
print(train_texts[3])
print('真实标签：')
print(y_train)

In [ ]:
#对测试集进行处理
test_label=test_csv['label'] #读取测试集中的'label'列
y_test=test_label.values #将'label'列由series结构转换为ndarray结构
test_texts = [] #二维列表，每个元素都是一条文本分词后的词语列表
for i in range(len(test_csv['review'])):
    test_texts.append(' '.join(review_to_wordlist(test_csv['review'][i]))) #逐个添加每条文本分词后的词语列表
print('评论内容：')
print(test_texts[3])
print('真实标签：')
print(y_test)

<h3>2.2将评论中划分的词语映射为数字</h3>
<h4>利用keras里的token实现，首先用Tokenizer的 fit_on_texts 方法学习出文本的字典，然后通过texts_to_sequences实现对应的单词和数字的映射，最后通过pad_sequences方法补成同样长度

In [ ]:
#建立词库：从所有评论中找出出现频率表最多的5000个词
token=tf.keras.preprocessing.text.Tokenizer(num_words=5000)
token.fit_on_texts(train_texts)
word_index = token.word_index #word_index字典将评论内容中的每一个词都赋予一个数字，数字越小表明该词出现频率越高

In [ ]:
#根据上面生成的字典word_index将每条评论的每个单词映射为一个数字
train_sequences=token.texts_to_sequences(train_texts) #得到训练集文本文字对应的数字列表
test_sequences=token.texts_to_sequences(test_texts) #得到测试集文本文字对应的数字列表

In [ ]:
#显示转换前的文字
print(train_texts[3])

In [ ]:
#显示转换后的数字列表
print(train_sequences[3])

In [ ]:
#确定训练集与测试集中的最长评论长度
reviews_lens = map(len,train_texts+test_texts) #确定最长评论词数
reviews_len_max=max(reviews_lens)
reviews_len_max

<h4>查看训练集中转化后的评论和标签</h4>

In [ ]:
#将每条评论都变为与最长评论相同长度的数字列表，不足的部分补0
#padding='post'需要补0时在序列结尾补；truncating='post'需要截断时从结尾截断；maxlen是序列最大长度
#还有一种方式,padding='pre'需要补0时在序列开头补；truncating='pre'需要截断时从开头截断
x_train=tf.keras.preprocessing.sequence.pad_sequences(train_sequences,padding='post',truncating='post',maxlen=reviews_len_max)
x_test=tf.keras.preprocessing.sequence.pad_sequences(test_sequences,padding='post',truncating='post',maxlen=reviews_len_max)

In [ ]:
print(x_train)
print(y_train)

<h3>2.3随机抽取构建训练集</h3>

In [ ]:
index = np.arange(len(train_texts)) #生成索引
np.random.shuffle(index) #打乱索引
print(x_train[index]) 
print(y_train[index])

<h2>3.构架LSTM模型</h2>
<h3>3.1使用tf.keras.models.Sequential()构建模型</h3>
<h4>用layer来搭建顺序模型</h4>

In [ ]:
model=tf.keras.models.Sequential([
    #嵌入层：将已经数字化的文本转化为向量
    #向量化目的：实现将词语嵌入多维矩阵，使得语义相近的词语，在空间距离上更接近
    #设置输出的词向量维度为64，输入维度5000，每条评论长度与最长评论相同
    tf.keras.layers.Embedding(output_dim=64,input_dim=5000,input_length=reviews_len_max),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), #双向封装器，用于对序列进行前向和后向计算
    tf.keras.layers.Dense(64, activation='relu'), #隐藏层，64个神经元，激活函数为relu函数
    #Dropout层以指定的丢弃概率随机丢弃上一层神经元
    tf.keras.layers.Dropout(0.2), #防止过拟合：随机选择丢弃20%神经元
    tf.keras.layers.Dense(1,activation='sigmoid') #输出层
])



<h3>3.2使用compile()配置训练算法</h3>

In [ ]:
#lr指学习率，epsilon指定一个较小的数代替0，防止实现过程除以0
#decay为学习率在每轮训练的衰减因子，取值范围[0,1]，0代表学习率在训练过程中保持不变
#经衰减后的学习率的计算公式为lr_i = lr_start * 1.0 / (1.0 + decay*i) , i为迭代周期，lr_start是lr的初始值

adam = tf.keras.optimizers.Adam(lr=0.0001,epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, #指定优化器——adam
              loss='binary_crossentropy', #指定损失函数——对数损失函数(针对二分类问题)
              metrics=['accuracy']) #设置模型检验的方法——准确度

<h2>4.训练模型</h2>
<h3>4.1使用fit()训练模型</h3>

<h1><strong>（可跳过训练直接加载训练数据）</strong></h1>

In [ ]:
#函数返回：训练过程的数据记录history
history = model.fit(x_train[index],y_train[index], #输入训练集的文字和标签
                    validation_split=0.3, #取训练集30%数据当验证集
                     epochs=10,batch_size=56,verbose=1,shuffle=True)
#verbose=0表示不在标准输出流输出日志信息，verbose=1表示输出进度条记录，verbose=2表示为每个epoch输出一行记录

In [ ]:
#训练耗时较长，训练完后保存一次数据
model.save_weights('./imdb-classify-lstm/finalmodel_weibo')

In [18]:
#恢复训练的数据
model.load_weights('./imdb-classify-lstm/finalmodel_weibo')

<h3>4.2训练过程可视化</h3>
<h4>绘制损失值变化过程</h4>

In [19]:
#trainRecord.history为字典对象，包含训练过程中的loss和测量指标等记录项
train_acc = history.history['accuracy'] #模型在训练集上的精度
val_acc = history.history['val_accuracy'] #模型在验证集上的精度
train_loss = history.history['loss'] #模型在训练集上的损失值
val_loss = history.history['val_loss'] #模型在验证集上的损失值

epochs=range(1,len(train_acc)+1) #设置横坐标epochs的计算方法
plt.figure(figsize=(9,7)) #设置图形的宽度和高度
#绘制一条折线，以训练轮数epochs作为x轴，训练集的损失值train_loss作为y轴，线条颜色为红色，线条标签为'Training Loss'
plt.plot(epochs,train_loss,'r',label='Training Loss') 
#绘制一条折线，以训练轮数epochs作为x轴，验证集的损失值val_loss作为y轴，线条颜色为蓝色，线条标签为'Validation Loss'
plt.plot(epochs,val_loss,'b',label='Validation Loss')
plt.title('Training and Validation Loss') #图形的标题
plt.xlabel('Epochs') #横坐标表示经过几轮迭代
plt.ylabel('Loss') #纵坐标表示损失值
plt.legend()
plt.show()

NameError: name 'history' is not defined

<h4>绘制准确度变化过程</h4>

In [ ]:
plt.clf()  #清除所有轴，保持窗口打开
plt.figure(figsize=(9,7)) #设置图形的宽度和高度
plt.plot(epochs,train_acc,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'b',label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs') #横坐标表示经过几轮迭代
plt.ylabel('Accuracy') #纵坐标表示准确度
plt.legend()
plt.show()

<h2>5.评估与预测</h2>
<h3>5.1使用evaluate()预测并评估测试集的预测结果</h3>

In [20]:
#输入数据和标签，输出损失值和准确度
test_loss,test_acc=model.evaluate(x_test,y_test,verbose=1)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

515/515 [==============================] - 18s 35ms/step - loss: 0.0968 - accuracy: 0.9687
Test Loss: 0.09676802158355713
Test Accuracy: 0.9687467217445374


<h3>5.2使用predict()对测试集进行预测</h3>

In [22]:
predictions=model.predict(x_test) #输入测试集，返回预测结果
predictions #显示

array([[0.9772314 ],
       [0.98703027],
       [0.99108505],
       ...,
       [0.00500336],
       [0.00137082],
       [0.00186062]], dtype=float32)

In [23]:
#将预测结果写入csv文件，包含文本内容，真实标签，预测标签
with open(r'.\data\weibo_prediction.csv', 'w',encoding='utf-8-sig',newline='') as csvfile:
    fieldnames = ['评论内容', '真实标签','预测标签']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(test_texts)):
        writer.writerow({'评论内容': test_texts[i], '真实标签': y_test[i],'预测标签': predictions[i][0]})

<h3>5.3保存并显示测试集的预测结果</h3>

In [24]:
#读取保存的csv文件并展示
df = pd.read_csv(r'.\data\weibo_prediction.csv',encoding='utf-8-sig')
df

,评论内容,真实标签,预测标签
0,更博 了 爆照 了 帅 的 呀 就是 越来越 爱 你 生快 傻 缺爱 你 爱 你 爱 你,1,0.977231
1,土耳其 的 事要 认真对待 哈哈 否则 直接 开除 很 是 细心 酒店 都 全部 啦,1,0.987030
2,姑娘 都 羡慕 你 呢 还有 招财猫 高兴 哈哈 小 学徒 一枚 等 着 明天 见 您 呢 ...,1,0.991085
3,美爱 你,1,0.502319
4,梦想 有 多 大 舞台 就 有 多 大 鼓掌,1,0.989366
...,...,...,...
23995,一 公里 不到 县 医院 那个 天桥 下右 拐 米 就 到 了 我 靠 这个 太 霸道 了 ...,0,0.008835
23996,今天 真冷 啊 难道 又 要 穿 棉袄 了 晕 今年 的 春天 真的 是 百变 莫测 啊 抓狂,0,0.004863
23997,最近 几天 就 没 停止 过 伤心,0,0.005003
23998,怒 很惨,0,0.001371


<h2>6.模型应用</h2>
<h3>6.1定义预测结果显示函数</h3>

In [25]:
def display_test_sentiment(text):
    #显示要预测的文本内容
    print(text)
    #数据规范化
    newtext=review_to_wordlist(text)
    #将文本转换为数字序列
    input_seq=token.texts_to_sequences([newtext])
    #将序列填充或截断为固定长度的序列
    pad_input_seq=tf.keras.preprocessing.sequence.pad_sequences(input_seq,
                                                   padding='post',
                                                   truncating='post',
                                                   maxlen=reviews_len_max)
    #使用训练好的模型进行预测
    pred=model.predict(pad_input_seq)
    print(pred[0][0])
    #打印预测值
    if pred[0][0]>0.5:
        preValue='positive'
    else:
        preValue='negtive'
    print('predict value:',preValue)

<h3>6.2对输入评论进行预测</h2>

In [26]:
review_text='''但是，最近我们发现有同学存在舞弊行为，有同学抄袭了别人的报告，有同学带着别人的报告在实验室做实验。对于这些行为，基础物理实验教学团队经讨论决定给予相关同学该实验0分的处理。希望我们所有同学以此为戒，从自身做起，恪守学术道德。
'''
display_test_sentiment(review_text)

但是，最近我们发现有同学存在舞弊行为，有同学抄袭了别人的报告，有同学带着别人的报告在实验室做实验。对于这些行为，基础物理实验教学团队经讨论决定给予相关同学该实验0分的处理。希望我们所有同学以此为戒，从自身做起，恪守学术道德。

0.14801186
predict value: negtive
